<a href="https://colab.research.google.com/github/Ryoya1012/Data-Analysis/blob/main/reflesh_file.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import gaussian_kde
import os  # Import os for file existence check

#sheet_name = '定常期'
#Tension = 1.7
parcent = 50
result_csv_path = f'F:\\New_test\\Traction_test\\Selection\\density_results_all(定常期50%).csv'
result = f'F:\\New_test\\Traction_test\\Selection\\density_results_all(定常期50%).png'

file_paths_group = [
    f'F:\\New_test\\Traction_test\\Selection\\250N\\density_results250.csv',
    f'F:\\New_test\\Traction_test\\Selection\\500N\\density_results500.csv',
    f'F:\\New_test\\Traction_test\\Selection\\750N\\density_results750.csv',
    f'F:\\New_test\\Traction_test\\Selection\\900N\\density_results900.csv',
    f'F:\\New_test\\Traction_test\\Selection\\1.1kN\\density_results1.1kN.csv',
    f'F:\\New_test\\Traction_test\\Selection\\1.2kN\\density_results1.2kN.csv',
    f'F:\\New_test\\Traction_test\\Selection\\1.3kN\\density_results1.3kN.csv',
    f'F:\\New_test\\Traction_test\\Selection\\1.4kN\\density_results1.4kN.csv',
    f'F:\\New_test\\Traction_test\\Selection\\1.5kN\\density_results1.5kN.csv',
    f'F:\\New_test\\Traction_test\\Selection\\1.6kN\\density_results1.6kN.csv',
    f'F:\\New_test\\Traction_test\\Selection\\1.7kN\\density_results1.7kN.csv',
]

all_file_paths = file_paths_group
df_list = []

# Load the data from the Excel files, skipping any missing files
for file_path in all_file_paths:
    try:
        if os.path.exists(file_path):  # Check if the file exists
            df = pd.read_csv(file_path)
            df_list.append(df[['Tension', 'Force']])
        else:
            print(f"File not found: {file_path}")
    except Exception as e:
        print(f"Error reading {file_path}: {e}")

# Combine all DataFrames into one if there are any valid files
if df_list:
    combined_df = pd.concat(df_list, ignore_index=True)

    # Extract Tension and Force
    x = combined_df['Tension']
    y = combined_df['Force']

    # Perform kernel density estimation
    xy = np.vstack([x, y])
    density = gaussian_kde(xy)(xy)

    # Determine the density threshold
    density_threshold = np.percentile(density, parcent)
    x_dense = x[density >= density_threshold]
    y_dense = y[density >= density_threshold]

    # Create a DataFrame for the results
    result_df = pd.DataFrame({'Tension': x_dense, 'Force': y_dense})

    # Save the results to a CSV file
    result_df.to_csv(result_csv_path, index=False)

    print(f"Results saved to {result_csv_path}")

    # Create a scatter plot of the filtered results
    plt.figure(figsize=(10, 6))
    plt.scatter(x_dense, y_dense, color='blue', alpha=0.5, label='Filtered Data Points')
    plt.title(f'Scatter Plot of Filtered Data (Tension:N)')
    plt.xlabel('Tension')
    plt.ylabel('Force')
    plt.grid(True)
    plt.legend()
    plt.savefig(result)
    plt.show()
else:
    print("No valid data files were found.")
